In [1]:
import torch
from torch import nn, optim

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets



In [2]:
batch_size = 32
learning_rate = 1e-2
num_epoches = 50

# 下载训练集 MNIST 手写数字训练集
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [3]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [12]:
class NN(nn.Module):
    def __init__(self,in_dim,n_hd_1,n_hd_2,n_hd_3,out_dim):
        super(NN,self).__init__()
        self.layer1 = nn.Linear(in_dim,n_hd_1)
        self.layer2 = nn.Linear(n_hd_1,n_hd_2)
        self.layer3 = nn.Linear(n_hd_2,n_hd_3)
        self.layer4 = nn.Linear(n_hd_3,out_dim)
        
    def forward(self,x):
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        x=self.layer4(x)
        return x

In [13]:
model = NN(28 * 28,300,500,200, 10)
if torch.cuda.is_available():
    model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [17]:
for epoch in range(num_epoches):
    print('epoch {}'.format(epoch + 1))
    print('*' * 10)
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):
        img, label = data
        img = img.view(img.size(0), -1)
        if torch.cuda.is_available():
            img = Variable(img).cuda()
            label = Variable(label).cuda()
        else:
            img = Variable(img)
            label = Variable(label)
        # 向前传播
        out = model(img)
        loss = criterion(out, label)
        running_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        running_acc += num_correct.data[0]
        # 向后传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 300 == 0:
            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(
                epoch + 1, num_epoches, running_loss / (batch_size * i),
                running_acc / (batch_size * i)))
    print('Finish {} epoch, Loss: {:.6f}, Acc: {:.6f}'.format(
        epoch + 1, running_loss / (len(train_dataset)), running_acc / (len(
            train_dataset))))
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    for data in test_loader:
        img, label = data
        img = img.view(img.size(0), -1)
        if torch.cuda.is_available():
            img = Variable(img, volatile=True).cuda()
            label = Variable(label, volatile=True).cuda()
        else:
            img = Variable(img, volatile=True)
            label = Variable(label, volatile=True)
        out = model(img)
        loss = criterion(out, label)
        eval_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        eval_acc += num_correct.data[0]
    print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(
        test_dataset)), eval_acc / (len(test_dataset))))
    print()


epoch 1
**********
[1/50] Loss: 0.248014, Acc: 0.928333
[1/50] Loss: 0.242669, Acc: 0.931406
[1/50] Loss: 0.240007, Acc: 0.932917
[1/50] Loss: 0.244687, Acc: 0.932109
[1/50] Loss: 0.245649, Acc: 0.932146
[1/50] Loss: 0.244803, Acc: 0.931684
Finish 1 epoch, Loss: 0.244544, Acc: 0.931700
Test Loss: 0.275337, Acc: 0.925500

epoch 2
**********
[2/50] Loss: 0.254368, Acc: 0.930521
[2/50] Loss: 0.248158, Acc: 0.931875
[2/50] Loss: 0.245143, Acc: 0.932778
[2/50] Loss: 0.243869, Acc: 0.932786
[2/50] Loss: 0.246560, Acc: 0.932375
[2/50] Loss: 0.245909, Acc: 0.932118
Finish 2 epoch, Loss: 0.244881, Acc: 0.932167
Test Loss: 0.274270, Acc: 0.924500

epoch 3
**********
[3/50] Loss: 0.235907, Acc: 0.933438
[3/50] Loss: 0.234455, Acc: 0.934167
[3/50] Loss: 0.239607, Acc: 0.932847
[3/50] Loss: 0.243758, Acc: 0.932083
[3/50] Loss: 0.244025, Acc: 0.932417
[3/50] Loss: 0.245334, Acc: 0.932031
Finish 3 epoch, Loss: 0.244782, Acc: 0.932283
Test Loss: 0.279766, Acc: 0.924000

epoch 4
**********
[4/50] Loss:

KeyboardInterrupt: 

Test Loss: 0.282118, Acc: 0.921500



In [16]:

# 保存模型
torch.save(model.state_dict(), './neural_network.pth')